In [ ]:
import requests
import networkx as nx
import matplotlib.pyplot as plt
import time

# Etherscan API密钥
api_key = 'API_KEY'

# 以太坊账户地址
address = 'Ether_Address'

# 构建API请求URL，获取最新的交易
url = f'https://api.etherscan.io/api?module=account&action=txlist&address={address}&sort=desc&apikey={api_key}'

# 发送GET请求获取交易数据
response = requests.get(url)

# 解析JSON响应
data = response.json()

# 获取所有交易
transactions = []
if data['status'] == '1':
    transactions = data['result']

# 按照时间戳排序交易
transactions.sort(key=lambda x: int(x['timeStamp']), reverse=True)

# 获取最后5个交易
transactions = transactions[:5]

# 创建一个空的有向图
G = nx.DiGraph()

# 打印每个交易
for tx in transactions:
    print(tx)

# 添加节点和边
for tx in transactions:
    sender = tx['from']
    receiver = tx['to']
    # 计算gas费用
    gas_price = int(tx['gasPrice'])  # 获取gas价格
    gas = int(tx['gas'])  # 获取gas数量
    gas_fee = gas * gas_price / 1e18 # 计算总gas费用
    print(gas_fee)
    timestamp = time.gmtime(int(tx['timeStamp']))
    timeofTranscation = time.strftime("%Y-%m-%d %H:%M:%S", timestamp)# 将Unix时间戳转换为日期和时间
    value = float(tx['value']) / 1e18  # 将以太币值转换为以太
    G.add_node(sender, type='sender', gas=gas_fee, timestamp = timeofTranscation, value=value)
    G.add_node(receiver, type='receiver', gas=gas_fee, timestamp = timeofTranscation, value=value)
    G.add_edge(sender, receiver)

# 绘制图谱
plt.figure(figsize=(15, 9))

# 设置节点位置
pos = nx.spring_layout(G, scale=0.9)  # 调整scale参数以减小图像尺寸

# 绘制节点，接受者红色
node_colors = {'sender': 'skyblue', 'receiver': 'salmon'}
for node, attr in G.nodes(data=True):
    nx.draw_networkx_nodes(G, pos, nodelist=[node], node_size=500, node_color=node_colors[attr['type']], label=None)

# 绘制边
nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.5)

# 绘制节点标签
nx.draw_networkx_labels(G, pos, font_size=8, font_weight='bold')

# 标记交易信息
for node, attr in G.nodes(data=True):
    if attr['type'] == 'sender':
        plt.text(pos[node][0], pos[node][1] + 0.1, f"Gas: {format(attr['gas'], '.8f')}\nTime: {attr['timestamp']}\nValue: {attr['value']:.8f}", ha='center', va='center')
    elif attr['type'] == 'receiver':
        plt.text(pos[node][0], pos[node][1] - 0.15, f"Gas: {format(attr['gas'], '.8f')}\nTime: {attr['timestamp']}\nValue: {attr['value']:.8f}", ha='center', va='center')

plt.title('Last 5 Ethereum Transactions Network of The Address')
plt.subplots_adjust(top=0.85) 
plt.axis('off')
plt.tight_layout()
plt.show()

